In [ ]:
import os
import sys

from unsloth import FastLanguageModel
import torch
from unsloth.chat_templates import get_chat_template, train_on_responses_only
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq, TrainerCallback
from unsloth import is_bfloat16_supported

import pandas as pd
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from dotenv import find_dotenv, load_dotenv

load_dotenv()

# Parameters + API Keys

In [ ]:
FINETUNING_DATASET_NAME="CPSC532/arxiv_qa_data"
CONFIG_NAME="2024NOV16_llama_3_1_8b_no_sources_in_question"
OUTPUT_MODEL_NAME="2024NOV16_llama_3_1_8b_no_sources_in_question"
BASE_MODEL_NAME="unsloth/Llama-3.2-3B-Instruct"

HF_TOKEN = os.getenv("HUGGINGFACE_API_KEY")

In [ ]:
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally! -> This affects memory during training, reduced from 16k
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


# Load Model + Chat Template

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_NAME, # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

def convert_to_messages_format(example):
    return [
        {"role": "user", "content": example['question']},
        {"role": "assistant", "content": example['answer']},
    ]

# Load Finetuning Dataset from HF + Pre-processing

In [ ]:
if HF_TOKEN is None:
  raise EnvironmentError("A hugggingface token is necessary!")
dataset_finetune = load_dataset(
                                FINETUNING_DATASET_NAME,
                                CONFIG_NAME,
                                split="train",
                                token=HF_TOKEN
                                )

In [ ]:
df = pd.DataFrame(dataset_finetune)

FILTER_OUT_STRINGS = ['no answer found', 'no text provided', 'no information provided']

# filter out rows that contain the strings in FILTER_OUT_STRINGS
for string in FILTER_OUT_STRINGS:
    df = df.loc[~df.answer.str.lower().str.contains(string)].reset_index(drop=True)


dataset_finetune = Dataset.from_pandas(df)
dataset_finetune = dataset_finetune.map(
                                        lambda x: {
                                            'conversations' : convert_to_messages_format(x)
                                            }
)
dataset_finetune = dataset_finetune.map(formatting_prompts_func, batched = True,)

In [ ]:
dataset_finetune['text'][0]

# Set Training Parameters + Model Training

In [ ]:
BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 4
PACKING = True

### Callbacks

In [ ]:
class GracefulStoppingCallback(TrainerCallback):
    def __init__(self):
        self.stop_requested = False
        
    def on_step_begin(self, args, state, control, **kwargs):
        if self.stop_requested:
            control.should_training_stop = True
            control.should_save = True
            
    def on_train_begin(self, args, state, control, **kwargs):
        import signal
        signal.signal(signal.SIGINT, self._signal_handler)
            
    def _signal_handler(self, sig, frame):
        print("\nStopping training gracefully...")
        self.stop_requested = True

### Trainer

In [ ]:

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_finetune,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 1,  # Affects memory usage
    packing = PACKING, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = BATCH_SIZE, # Affects memory usage
        gradient_accumulation_steps = GRADIENT_ACCUMULATION_STEPS,
        warmup_steps = 5,
        num_train_epochs = 5, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none"
    ),
    callbacks=[GracefulStoppingCallback]
)

In [ ]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

# Training Results

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
training_df = pd.DataFrame(trainer.state.log_history)

import sys
if not 'google.colab' in sys.modules:
  from helpers import create_training_plots
  fig = create_training_plots(training_df)
  fig.show()
  training_df.to_csv(f"training_logs/{OUTPUT_MODEL_NAME}.csv", index = False)

# Save to HF

In [ ]:
model.push_to_hub_gguf(
        f"CPSC532/{OUTPUT_MODEL_NAME}",
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m", "f16", "q4_0", "not_quantized"],
        token = HF_TOKEN
    )